In [1]:
import argparse
import numpy as np
import random
import argparse
import os
import time
import math
import sys
import json
import csv
import pandas as pd

import torch
from torch.autograd import Variable

from sklearn.metrics.pairwise import euclidean_distances, cosine_distances

from models import load_models_decode, generate
from utils import to_gpu, Corpus, batchify, train_ngram_lm, get_ppl

In [21]:
# load_path='output/out1102'
# load_path='output/out_twitter1103'
load_path='output/out1106_komoran'

In [22]:
model_args, idx2word, autoencoder, gan_gen, gan_disc, word2idx= load_models_decode(load_path)

Loading models fromoutput/out1106_komoran


In [8]:
vocab2=idx2word
vocab2

{0: '<pad>',
 1: '<sos>',
 2: '<eos>',
 3: '<oov>',
 4: '가',
 5: '가.',
 6: '가가',
 7: '가거나',
 8: '가게',
 9: '가게가도',
 10: '가게나',
 11: '가게되면',
 12: '가게되었으며',
 13: '가게마다',
 14: '가게에',
 15: '가게에서',
 16: '가게에서나',
 17: '가게주인이주어서.',
 18: '가격',
 19: '가격.',
 20: '가격과',
 21: '가격까지',
 22: '가격대',
 23: '가격대가',
 24: '가격대가있지만',
 25: '가격대도',
 26: '가격대로',
 27: '가격대비',
 28: '가격대에',
 29: '가격대와',
 30: '가격대중성.',
 31: '가격도',
 32: '가격도싸고',
 33: '가격면에서',
 34: '가격보다도',
 35: '가격부담없이',
 36: '가격에',
 37: '가격에다',
 38: '가격에서',
 39: '가격은',
 40: '가격을',
 41: '가격이',
 42: '가격이랑',
 43: '가격이싸고',
 44: '가격저렴.',
 45: '가격저렴하고',
 46: '가격적으로나',
 47: '가격적으로도',
 48: '가격적인',
 49: '가격차이',
 50: '가격치곤',
 51: '가고',
 52: '가공공법',
 53: '가기',
 54: '가까운',
 55: '가까운데서',
 56: '가까운맛.',
 57: '가까움',
 58: '가까움.',
 59: '가까이서',
 60: '가까이에서',
 61: '가깝게',
 62: '가깝다고',
 63: '가끔',
 64: '가끔가다',
 65: '가끔식',
 66: '가끔씩',
 67: '가나',
 68: '가는',
 69: '가는거같다.',
 70: '가는구간에도',
 71: '가는데',
 72: '가는듯한',
 73: '가능.',
 74: '가능하고',
 75: '가능하다',
 76: '가능하다.',
 77: '가능하며

In [81]:
word2idx

{'<pad>': 0,
 '<sos>': 1,
 '<eos>': 2,
 '<oov>': 3,
 'ㄴ': 4,
 'ㄴ.': 5,
 'ㄴ가': 6,
 'ㄴ가.': 7,
 'ㄴ가요.': 8,
 'ㄴ걸': 9,
 'ㄴ고': 10,
 'ㄴ다': 11,
 'ㄴ다.': 12,
 'ㄴ다거나': 13,
 'ㄴ다고': 14,
 'ㄴ다기': 15,
 'ㄴ다는': 16,
 'ㄴ다면': 17,
 'ㄴ데': 18,
 'ㄴ데요': 19,
 'ㄴ듯': 20,
 'ㄴ듯.': 21,
 'ㄴ지': 22,
 'ㄴ지.': 23,
 'ㄹ': 24,
 'ㄹ.': 25,
 'ㄹ걸': 26,
 'ㄹ까': 27,
 'ㄹ까.': 28,
 'ㄹ까요': 29,
 'ㄹ까요.': 30,
 'ㄹ려고': 31,
 'ㄹ지': 32,
 'ㄹ지라도': 33,
 'ㅁ': 34,
 'ㅁ.': 35,
 'ㅂ니다': 36,
 'ㅂ니다.': 37,
 'ㅂ사': 38,
 '가': 39,
 '가.': 40,
 '가감.': 41,
 '가게': 42,
 '가격': 43,
 '가격.': 44,
 '가공': 45,
 '가까이': 46,
 '가깝': 47,
 '가끔': 48,
 '가끔가다': 49,
 '가능': 50,
 '가능.': 51,
 '가득': 52,
 '가득.': 53,
 '가라앉히': 54,
 '가량': 55,
 '가루': 56,
 '가볍': 57,
 '가수': 58,
 '가운데': 59,
 '가장': 60,
 '가제': 61,
 '가족': 62,
 '가지': 63,
 '가치': 64,
 '각': 65,
 '각인': 66,
 '각지': 67,
 '간': 68,
 '간간히': 69,
 '간결': 70,
 '간단': 71,
 '간단명료': 72,
 '간단히': 73,
 '간편': 74,
 '간혹': 75,
 '갈': 76,
 '갈리': 77,
 '갈비': 78,
 '갈색': 79,
 '갈수록': 80,
 '갈증': 81,
 '감': 82,
 '감.': 83,
 '감각': 84,
 '감각.': 85,
 '감귤': 86,
 '감기': 87,

In [400]:
word2idx['흔해서.']

10899

In [401]:
vocab2[10899]

'흔해서.'

In [9]:
model_args

{'N': 5,
 'arch_d': '300-300',
 'arch_g': '300-300',
 'batch_size': 5,
 'beta1': 0.9,
 'clip': 1,
 'cuda': True,
 'data_path': 'data_korean',
 'dropout': 0.0,
 'emsize': 300,
 'enc_grad_norm': True,
 'epochs': 49,
 'gan_clamp': 0.01,
 'gan_toenc': -0.01,
 'hidden_init': False,
 'kenlm_path': '../Data/kenlm',
 'log_interval': 200,
 'lowercase': False,
 'lr_ae': 1,
 'lr_gan_d': 1e-05,
 'lr_gan_g': 5e-05,
 'maxlen': 30,
 'min_epochs': 50,
 'nhidden': 300,
 'niters_ae': 1,
 'niters_gan_d': 5,
 'niters_gan_g': 1,
 'niters_gan_schedule': '2-4-6',
 'nlayers': 1,
 'no_earlystopping': False,
 'noise_anneal': 0.995,
 'noise_radius': 0.2,
 'ntokens': 10919,
 'outf': 'out1102',
 'patience': 5,
 'sample': False,
 'seed': 1111,
 'temp': 1,
 'vocab_size': 11000,
 'z_size': 100}

# ---------전처리

In [266]:
import re

In [254]:
file = 'codedata/rawinput.csv'
rawdata = pd.read_csv(file, names = None, encoding='CP949')
rawdoc = rawdata['응답값']

In [265]:
rawdoc1 = [" ".join(re.findall('([가-힣]+)', row))+"." for row in rawdoc]
rawdoc1
with open("train.txt", 'w') as f:
    for s in rawdoc1:
        f.write(str(s) + '\n')

# -----------------------------

In [2]:
from konlpy.tag import Komoran

In [3]:
pos_tagger = Komoran()

In [4]:
def tokenize_komoran(doc):
    # norm, stem은 optional
    return [x for x, t in pos_tagger.pos(doc)]

In [7]:
token = []
for row in doc :
    token.append(' '.join(k for k in tokenize_komoran(row)))

In [30]:
data['raw_doc']= pd.DataFrame(token)[0]
tokendoc = data['raw_doc']

In [16]:
data

,clusterno,Unnamed: 1,docNo,raw_doc
0,0,0,8828,시원 하 다
1,0,1,2951,시원 하 ㄴ 맛
2,0,2,8632,시원
3,0,3,8448,ㅈ다른 것 보다 더 시원 하 ㄴ 맛 이 나 ㄴ다
4,0,4,10013,"병 을 딸 때 시원 하 게 펑 하 는 소리 가 나 고 , 맛 도 좋 은 것 같 다"
5,0,5,5086,시원 하 고 탁 쏘 는 맛 이 봏아서
6,0,6,9394,시원 하 ㅁ
7,0,7,8762,시원 하 아서
8,0,8,5763,시원 하 ㄴ 맛
9,0,9,7968,맛 이 시원 하 다


# ------------------

In [16]:
path_raw = 'codedata/clusterdata.csv'
maxlen=100

In [5]:
path_raw = 'codedata/DBSCAN_result_Topic_refined.csv'
maxlen=100

In [32]:
data = pd.read_csv(path_raw, names = None,encoding='cp949')
clusternum=data['clusterno']
docno = data['docNo']
rawdoc = data['raw_doc']

In [35]:
groupby_cluster = rawdoc.groupby([clusternum])
groupby_cluster_token = tokendoc.groupby([clusternum])

In [36]:
cluster_doc=groupby_cluster_token.apply(lambda x: x.tolist()).to_dict()
cluster_doc_rawdata=groupby_cluster.apply(lambda x: x.tolist()).to_dict()

In [85]:
cluster_doc

{0: ['시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 아서.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원.',
  '시원 하 아서.',
  '시원 하 아서.',
  '시원 하 다.',
  '시원 하 다.',
  '시원 하 다.',
  '시원 하 아서.',
  '시원.',
  '시원 하 다.',
  '시원 하 아서.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 다.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 다.',
  '시원 하 ㅁ.',
  '시원 하 어요 ~.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 ㅁ.',
  '시원 하 아서.',
  '시원 하 아서

In [23]:
vocab = word2idx
ntokens = len(vocab)
def doc_to_line(cluster_doc, vocab):

    dropped = 0
    linecount = 0
    lines = []
    for line in cluster_doc:
        linecount += 1        
#         if self.lowercase:
#             words = line[:-1].lower().strip().split(" ")
#         else:
#             words = line[:-1].strip().split(" ")
        words = line[:-1].strip().split(" ")
        
        if len(words) > maxlen:
            dropped += 1
            continue
        words = ['<sos>'] + words
        words += ['<eos>']

        # vectorize

        unk_idx = vocab['<oov>']
        indices = [vocab[w] if w in vocab else unk_idx for w in words]

#         print(indices)
        lines.append(indices)
        
    return lines, vocab

In [24]:
def make_code(lines):
    eval_batch_size = 1
    test_data = batchify(lines, eval_batch_size, shuffle=False)
     
    code = []
    for i, batch in enumerate(test_data):
            source, target, lengths = batch
    #         source = to_gpu(True, Variable(source, volatile=True))
    #         target = to_gpu(True, Variable(target, volatile=True))
    #         mask = target.gt(0)
    #         masked_target = target.masked_select(mask)
    #         # examples x ntokens
    #         output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)

            # output: batch x seq_len x ntokens
            output = autoencoder.encode(Variable(source), lengths, noise=False)
#             print(output)
            code.append(output)
    return code

In [45]:
title={}
title_raw={}
cluster_title=[]
for c in cluster_doc.keys() : 
    lines, vocab = doc_to_line(cluster_doc[c], vocab)
    code = make_code(lines)

    code_cat=torch.cat(code,0)
    
    #code_feature extract(절대값 0.1미만 0으로)
#     code_cat_np = code_cat.data.numpy().squeeze()
#     temp = np.copy(code_cat_np)
#     temp[np.where(np.abs(temp)<feature_range)]=0
#     code_feature=torch.FloatTensor(temp).view(-1,300)
    
    #code 평균
    code_mean=torch.mean(code_cat,0).view(1,-1)
    
    #code 평균과 길이가 최소인 vector 구하기
    input1 = code_cat.data.numpy()
    mean_np = code_mean.data.numpy()
#     l2 = euclidean_distances(input1, mean_np).squeeze()
    l2 = cosine_distances(input1, mean_np).squeeze()
        
    title[c]=cluster_doc[c][np.argmin(l2)]
    title_raw[c]=cluster_doc_rawdata[c][np.argmin(l2)]

In [46]:
title

{'-1': '하이트 는 우선 제일 흔하 게 접하 ㄹ 수 있 는 맥주 브랜드 이 다 . 광고 에서 도 보 면 유명 연예인 들 을 기용 하 기 때문 에 먼저 눈 이 가 고 , 신뢰 하 ㄹ 만 하 ㄴ 브랜드 이 ㄹ 것 같 다는 생각 을 들 게 만들 기 때문 에 하이트 를 가장 선호 하 는 편 이 다 .',
 '0': '시원 하 다',
 '1': '맛 이 깔끔 하 다',
 '10': '고급 스럽 ㅁ',
 '100': '하이트 가 맥주 맛 이 제일 낫 다 . 익숙 하 아서 그렇 ㄴ지 는 모르 아도 다른 수입 맥주 들 은 기분 전환 겸 가끔가다 한번 씩 먹 는데 맥주 살 때 주저 하 지 않 고 살 는 건 하이트 뿐 이 ㄴ 거 같 다 .',
 '101': '부드럽 ㄴ 느낌이 좋아 서',
 '102': '달콤 하 ㄴ 맛 이 좋 습니다',
 '103': '눈 에 한 번 에 들어오 ㅁ',
 '104': '세련 되 고 젊 다',
 '105': '이름 도 세련 되 있 고 상표 도 깔끔 하 다 .',
 '106': '맛 이 있 고 내 취향 에 맞 아서',
 '107': '여러 종류 의 맥주 를 마시 어 보 았 지만 가장 처음 접하 았었 고 오 라 하 았 동안 마시 어 오 ㄴ 맥주 이 라서 좋아하 ㅂ니다 . 맛 도 물론 깔끔 하 고요 . 쓰 ㄴ 맛 이 나 지 않 아 좋 습니다 .',
 '108': '텔레비젼 광고 를 보 고 . 시원 하 고 상쾌 하 ㄹ 것 같 은 맛 때문',
 '109': '하이네켄 을 처음 마시 었 을 때 의 추억 이 가장 좋 다 . ( 당시 환경 때문 )',
 '11': '깔끔 하 고 시원 하 ㅁ',
 '110': '예전 부터 자주 먹 는 맥주 이 다 . 다른 맥주 는 대체 적 으로 안 먹 음 . 특히 소주 랑 믹스 하 아서 먹 으면 아주 좋 다 .',
 '111': '가장 보편 적 이 라서',
 '112': '다른 맥주 와 차별 화 되 ㄴ 맛 .',
 '113': '강원도 홍천 공장 에 방문 하 았 을 때 먹 어 보 았 음',
 '114': '습관 적 으로',


In [47]:
title_raw

{'-1': '하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다.  광고에서도 보면 유명 연예인들을 기용하기 때문에 먼저 눈이 가고, 신뢰할만한 브랜드일 것 같다는 생각을 들게 만들기 때문에 하이트를 가장 선호하는 편이다.',
 '0': '시원하다',
 '1': '맛이 깔끔하다',
 '10': '고급스럼',
 '100': '하이트가 맥주맛이 제일 낫다.  익숙해서 그런지는 몰라도 다른 수입맥주들은 기분전환겸 가끔가다 한번씩 먹는데 맥주살때 주저하지않고 사는건 하이트뿐인거같다.',
 '101': '부드러운 느낌이 좋아서',
 '102': '달콤한 맛이 좋습니다',
 '103': '눈에 한 번에 들어옴',
 '104': '세련되고 젊다',
 '105': '이름도 세련되있고 상표도 깔끔하다.',
 '106': '맛이 있고 내 취향에 맞아서',
 '107': '여러 종류의 맥주를 마셔봤지만 가장 처음 접했었고 오랬동안 마셔온 맥주라서 좋아합니다. 맛도 물론 깔끔하고요.쓴 맛이 나지 않아 좋습니다.',
 '108': '텔레비젼 광고를 보고 . 시원하고 상쾌할 것 같은 맛 때문',
 '109': '하이네켄을 처음 마셨을 때의 추억이 가장 좋다.(당시 환경 때문)',
 '11': '깔끔하고 시원함',
 '110': '예전부터 자주 먹는 맥주이다.다른 맥주는 대체적으로 안먹음.  특히 소주랑 믹스해서 먹으면 아주 좋다.',
 '111': '가장 보편적이라서',
 '112': '다른 맥주와 차별화 된 맛.',
 '113': '강원도 홍천 공장에 방문했을때 먹어봤음',
 '114': '습관적으로',
 '115': '카프리를 병으로 먹으면 톡쏘는 맛이 좋아서 배부르지 않으면서도 한병으로 기분좋게 마실수 있는 점이 좋습니다.',
 '116': '어느 가게가도 있으니까~',
 '117': '레몬의 맛과 맥주의 맛이 잘 어우러져있어서',
 '118': '가장 기본적인 맥주맛이라서',
 '119': '맥주 맛이 풍부하고, 좀더 시원한 느낌이 들어서 입니다.',
 '12': '자주

In [48]:
df = pd.DataFrame([title_raw,cluster_doc]).T
df['clusterNo'] = df.index

In [49]:
df

,0,1,clusterNo
-1,하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다. 광고에서도 보면 유명 ...,[청량감 이 느끼 어 지 는 초록 병 과 이쁘 ㄴ 병뚜껑 전체 적 으로 세련 되 ㄴ...,-1
0,시원하다,"[시원 하 다, 시원 하 ㄴ 맛, 시원, ㅈ다른 것 보다 더 시원 하 ㄴ 맛 이 나...",0
1,맛이 깔끔하다,"[깔끔, 끝 맛 이 깔끔 하 아서, 깔끔 하 ㅁ, 맛 이 깔끔 하 아서, 깔끔 하 ...",1
10,고급스럼,"[고급 스럽 ㅁ, 고급 스럽 ㅁ, 맛 의 풍미 가 깊 다, 풍미 작 려 ㄹ, 흑 맥...",10
100,하이트가 맥주맛이 제일 낫다. 익숙해서 그런지는 몰라도 다른 수입맥주들은 기분전환...,[아사히 같 은 경우 에 는 제 가 수입 맥주 중 에 처음 으로 즐기 었 던 맥주 ...,100
101,부드러운 느낌이 좋아서,"[레몬 이랑 함께 먹 으면 상쾌 하 고 개인 적 으로 상큼 하 ㄴ 느낌이 좋아 요,...",101
102,달콤한 맛이 좋습니다,"[음료수 맛 이 나서 어 맛있 어서, 다양 하 ㄴ 맥주 맛 이 좋 음, 맛 이 달콤...",102
103,눈에 한 번에 들어옴,"[옛날 풍 .... 그런 느낌 . . 고풍 스럽 다고 나 하 ㄹ까 .. 음, 깔끔 ...",103
104,세련되고 젊다,"[다른 맥주 상표 보다 세련 되 다, 개인 적 으로 맥주 맛 이 괜찮 고 디자인 이...",104
105,이름도 세련되있고 상표도 깔끔하다.,"[그냥 이름 이 많이 알리 어 지 어서 , ,, 이름 도 세련 되 있 고 상표 도 ...",105


In [50]:
rows = []
_ = df.apply(lambda row: [rows.append([row['clusterNo'],nn,row[0]]) 
                         for nn in row[1]], axis=1)

In [51]:
df_new = pd.DataFrame(rows, columns=['clusterNo','raw_docs','title'])
df_new

,clusterNo,raw_docs,title
0,-1,청량감 이 느끼 어 지 는 초록 병 과 이쁘 ㄴ 병뚜껑 전체 적 으로 세련 되 ㄴ ...,하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다. 광고에서도 보면 유명 ...
1,-1,우리나라 에서 기본 적 이 ㄴ 맥주 이 고 시중 에 가장 가깝 게 접하 ㄹ 수 있 ...,하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다. 광고에서도 보면 유명 ...
2,-1,그냥 제일 처음 나오 았 을 때 첫인상 이 좋 았 고 역동 적 이 ㄴ 상표 가 좋 ...,하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다. 광고에서도 보면 유명 ...
3,-1,브랜드 가 마음 에 들 고 라벨 디자인 이 깔끔 하 고 세련 되 며 먹 는 한국 맥...,하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다. 광고에서도 보면 유명 ...
4,-1,동네 슈퍼 덜 ㄴ 편의점 이 던 어디 서 나 쉽 게 살 ㄹ 수 있 고 먹 기 에 부...,하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다. 광고에서도 보면 유명 ...
5,-1,내 기호 에 맞 다,하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다. 광고에서도 보면 유명 ...
6,-1,현재 시판 되 고 있 는 맥주 중 에서 유일 하 게 100 % 보리 맥주 이 고 식...,하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다. 광고에서도 보면 유명 ...
7,-1,청량감 과 톡 쏘 는 맛 이 약간 떨어지 기 ㄴ 하지만 기존 의 카스 보다 칼로리 ...,하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다. 광고에서도 보면 유명 ...
8,-1,진하 ㄴ 보리 향 과 혀 에 감기 는 맥주 맛 이 입맛 에 맞 기 떄문이다.,하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다. 광고에서도 보면 유명 ...
9,-1,이름 도 짧 아서 눈 에 확 띠 고 노란색 과 파란색 의 조화 로 보 기 도 좋 다,하이트는 우선 제일 흔하게 접할 수 있는 맥주 브랜드이다. 광고에서도 보면 유명 ...


In [52]:
df_new.to_csv('topicdbscan_komoran_cosine_center_vector_title.csv',sep=',',encoding='CP949')

# ----------------------------

In [365]:
feature_range=0.4


cluster_title=[]
for c in cluster_doc.keys() : 
    lines, vocab = doc_to_line(cluster_doc[c], vocab)
    code = make_code(lines)

    code_cat=torch.cat(code,0)
    
    #code_feature extract(절대값 0.1미만 0으로)
    code_cat_np = code_cat.data.numpy().squeeze()
    temp = np.copy(code_cat_np)
    temp[np.where(np.abs(temp)<feature_range)]=0
    code_feature=torch.FloatTensor(temp).view(-1,300)
    
    #code 평균
    code_mean=torch.mean(code_feature,0).view(1,-1) 

    max_indices = autoencoder.generate_code(code_mean, maxlen)
    max_indices = max_indices.data.cpu().numpy().squeeze()
    

    words = [vocab2[x] for x in max_indices.squeeze()]
#     print(words)
    # truncate sentences to first occurrence of <eos>
    truncated_sent = []
    for w in words:
        if w != '<eos>':
            truncated_sent.append(w)
        else:
            break
    sent = " ".join(truncated_sent)
    cluster_title.append(sent)

In [358]:
len(cluster_title)

536

In [366]:
resultdoc={}
for i in range(len(cluster_title)):
    resultdoc[i]=cluster_title[i]

In [352]:
resultdoc

{0: '하하. 청량감 향.',
 1: '맛 이 깔끔해서 풍부하다.',
 2: '부드러움 부드럽',
 3: '맛있어 요 부드러워',
 4: '목 넘김 이 부드럽 고 독특해서.',
 5: '톡 쏘는 맛 이 좋 다 그리고 탄산.',
 6: '편안하다. 가 있다.',
 7: '맛 이 너무 부드럽 아요.',
 8: '내 입맛 에 맛 잇어 서 익숙해서.',
 9: '맛 이 좋 아.',
 10: '산토리 맥주 가 대체 적 인 맛 에 끌림.',
 11: '목 넘김 이 좋 다 선호.',
 12: '맛 이 강하고 진하다 풍부함.',
 13: '그냥 좋 았 습니다.',
 14: '뒷맛 이 깔끔하고 풍부하다.',
 15: '시원하고 맛 도 가벼 움.',
 16: '맛 이 좋 고 색감',
 17: '맥주 가 맛 잇어 좋 아.',
 18: '목 넘김 이 좋 고 시원한 뒷맛.',
 19: '깔끔하고 맛 이 순하고 가벼워',
 20: '부드럽 고 진한 것 같고 고소함.',
 21: '거품 이 풍부해서 부드럽 고 맛 잇다.',
 22: '흑 맥주 를 좋아하고 기네스 가 더 훌륭함.',
 23: '맛 과 향 이 좋 다 독특해서.',
 24: '순하고 맛 이 너무 쓰',
 25: '상큼 한 맛 에 대한 고소함.',
 26: '쏘는 맛 이 인상 에 다',
 27: '맛 이 진하다 그리고 풍부함.',
 28: '가장 대중 적 이고 있으',
 29: '톡 쏘는 맛 이 좋 다 그리고 가격 이 높다.',
 30: '보리 의 맛 이 있어 서 시원한 맛 이라서.',
 31: '톡소 는 맛 이 좋 으며 선호.',
 32: '맛 이 산토리 와 다른 맛 이 일품.',
 33: '고급스러 움 병 색깔',
 34: '가격 도 저렴 하고 국산 맛 이 나 으니까.',
 35: '내 입맛 에 맛 는 그냥 그냥 먹음.',
 36: '버드와이저 맥주 보다 맛있어 보이 게 됨.',
 37: '자주 마시는 것 기 떄문 에 익숙하다.',
 38: '시원한 느낌 보다 시원한 느낌 이라서.',
 39: '익숙해서 부드럽 고 상쾌한

In [367]:
df = pd.DataFrame([resultdoc,cluster_doc]).T
df['clusterNo'] = df.index

In [361]:
df

,0,1,clusterNo
0,강예빈 의 청량감 참신 한 게 맛있어,"[시원해서., 시원함., 시원하다., 시원함., 시원해서., 시원함., 시원하다.,...",0
1,깔끔한 맛 이 강하지 고 세계,"[맛 이 깔끔하다., 맛 이 깔끔함., 맛 이 깔끔함., 깔끔한 맛., 맛 이 깔끔...",1
2,너무 목 넘김 또한 아주 좋아짐.,"[부드럽 다., 부드러움., 부드러움., 부드러움., 부드러움., 부드럽 다., 부...",2
3,맛있어 요 거품 에 적당한 것,"[맛있다., 맛있 음., 맛있어 서., 맛있어 서요., 맛있어 요., 맛있다., 맛...",3
4,목 넘김 이 부드럽 고 향 이 독특해서.,"[부드러운 목 넘김 이 좋 다 맛 도 부드러워 좋 다., 목 넘김 이 좋 고 부드럽...",4
5,톡 쏘는 맛 이 좋 다 그리고 톡 쏜다.,"[톡 쏘는 맛., 톡 쏘아 서., 맛 이 좋 음 톡 쏨., 톡 쏘는 맛 이 좋 다....",5
6,구하기 가 지금 깔 금하고 깨끗한 게 친숙함.,"[깔끔하다., 깔끔해 요., 깔끔함., 깔끔함., 깔끔해서., 깔끔해서., 깔끔하 ...",6
7,맛 이 너무 부드럽 다 가격 이.,"[부드러운 맛., 맛 이 부드러워 서., 부드러운 맛., 맛 이 부드럽 다., 맛 ...",7
8,맛 이 입맛 에 맞 음 고소함.,"[맛 이 입맛 에 맞아 서., 내 입맛 에 맞아 서., 내 입맛 에 맞 음., 맛 ...",8
9,맛 이 좋 다 풍부함.,"[맛 이 좋 다., 맛 이 좋 아서., 맛 이 좋 아서., 맛 이 좋 아서., 맛 ...",9


In [284]:
df

,0,1,clusterNo
0,시원 하 ㅁ 맛있 어요.,"[시원 하 아서., 시원 하 ㅁ., 시원 하 다., 시원 하 ㅁ., 시원 하 아서....",0
1,맛 이 깔끔 하 았 아.,"[맛 이 깔끔 하 다., 맛 이 깔끔 하 ㅁ., 맛 이 깔끔 하 ㅁ., 깔끔 하 ㄴ...",1
2,부드럽 ㅁ 아.,"[부드럽 다., 부드럽 ㅁ., 부드럽 ㅁ., 부드럽 ㅁ., 부드럽 ㅁ., 부드럽 다...",2
3,맛있 으니까.,"[맛있 다., 맛있 음., 맛있 어서., 맛있 어서요., 맛있 어요., 맛있 다.,...",3
4,목 넘기 ㅁ 이 부드럽 다 아.,"[부드럽 ㄴ 목 넘기 ㅁ 이 좋 다 맛 도 부드럽 어 좋 다., 목 넘기 ㅁ 이 좋...",4
5,톡 쏘 는 맛 순 하 아서 좋 아요.,"[톡 쏘 는 맛., 톡 쏘 아서., 맛 이 좋 음 톡 쏘 ㅁ., 톡 쏘 는 맛 이 ...",5
6,깔끔 하 아서 아.,"[깔끔 하 다., 깔끔 하 어요., 깔끔 하 ㅁ., 깔끔 하 ㅁ., 깔끔 하 아서....",6
7,부드럽 ㄴ 맛 이 담백.,"[부드럽 ㄴ 맛., 맛 이 부드럽 어서., 부드럽 ㄴ 맛., 맛 이 부드럽 다., ...",7
8,입맛 에 가장 맞 아서 아.,"[맛 이 입맛 에 맞 아서., 내 입맛 에 맞 아서., 내 입맛 에 맞 음., 맛 ...",8
9,맛 이 아서 좋 아.,"[맛 이 좋 다., 맛 이 좋 아서., 맛 이 좋 아서., 맛 이 좋 아서., 맛 ...",9


In [368]:
rows = []
_ = df.apply(lambda row: [rows.append([row['clusterNo'],row[0],  nn]) 
                         for nn in row[1]], axis=1)

In [369]:
df_new = pd.DataFrame(rows, columns=['clusterNo','title','raw_docs'])
df_new

,clusterNo,title,raw_docs
0,0,뒷 별 모양 이 맘 에 들어서 남자,시원해서.
1,0,뒷 별 모양 이 맘 에 들어서 남자,시원함.
2,0,뒷 별 모양 이 맘 에 들어서 남자,시원하다.
3,0,뒷 별 모양 이 맘 에 들어서 남자,시원함.
4,0,뒷 별 모양 이 맘 에 들어서 남자,시원해서.
5,0,뒷 별 모양 이 맘 에 들어서 남자,시원함.
6,0,뒷 별 모양 이 맘 에 들어서 남자,시원하다.
7,0,뒷 별 모양 이 맘 에 들어서 남자,시원함.
8,0,뒷 별 모양 이 맘 에 들어서 남자,시원해서.
9,0,뒷 별 모양 이 맘 에 들어서 남자,시원함.


In [370]:
df_new.to_csv('1114_twitter_generate_title_result_raw_0.4.csv',sep=',',encoding='CP949')

# ----------------------------code_vec

In [442]:
cluster_title=[]
code_vec=[]
l=0
for c in cluster_doc.keys() : 
    lines, vocab= doc_to_line(cluster_doc[c], vocab)
    eval_batch_size = 1
    
    test_data = batchify(lines, eval_batch_size, shuffle=False)
    
    ntokens = len(vocab)
    for i, batch in enumerate(test_data):
        
        source, target, lengths = batch
    #         source = to_gpu(True, Variable(source, volatile=True))
    #         target = to_gpu(True, Variable(target, volatile=True))
    #         mask = target.gt(0)
    #         masked_target = target.masked_select(mask)
    #         # examples x ntokens
    #         output_mask = mask.unsqueeze(1).expand(mask.size(0), ntokens)
        indices = Variable(source)
            # output: batch x seq_len x ntokens
        output = autoencoder.encode(indices, lengths, noise=False)
        
        code_vec.extend(output.data.numpy().tolist())
    l+=len(lines)

In [443]:
len(code_vec)

8524

In [444]:
df= pd.DataFrame(code_vec)
df1 = pd.concat([clusternum,doc,df], axis=1)
df1.to_csv('cluster_code_vec_raw_1115.csv', index=False)